# Libraries

In [200]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [201]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [202]:
%run Data_preprocessing_v1.ipynb

************************************************************
There are 6 set of X
X_train_selected_t1, X_train_selected_t2, X_valid_selected_t1,X_valid_selected_t2,X_test_selected_t1,X_test_selected_t2
------------------------------------------------------------
Normalized version
X_train_selected_t1_norm, X_train_selected_t2_norm, X_valid_selected_t1_norm,X_valid_selected_t2_norm,X_test_selected_t1_norm,X_test_selected_t2_norm
------------------------------------------------------------
There are 4 set of Y
y_train_t1, y_train_t2, y_valid_t1, y_valid_t2
when training, please use: 'y_train_t1_value,y_train_t2_value,y_valid_t1_value,y_valid_t2_value'
************************************************************


In [203]:
print(X_train_selected_t2_norm.shape)
print(X_valid_selected_t2_norm.shape)
print(X_test_selected_t2_norm.shape)

(16760, 54)
(2394, 54)
(4790, 54)


# Hyperparameters

In [204]:
input_size = 54
sequence_length = 1
output_size = 1 # live or dead
num_layers = 3
hidden_size=30
learning_rate = 0.001
batch_size = 32
num_epochs = 500
l2_lamb = 3.736842105263158e-05

# Create JusNN

In [205]:
class JusNN(nn.Module):
    def __init__(self, input_size, num_layers, output_size):
        super(JusNN, self).__init__()
        
        self.convlayer = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=6, kernel_size=4, stride=1, padding=0),
            nn.AvgPool1d(kernel_size=4,stride=3),
            nn.Conv1d(in_channels=6, out_channels=1, kernel_size=4, stride=1, padding=0),
            nn.ReLU(),
        )
        
        self.fclayers = nn.Sequential(
            nn.Linear(input_size, 3),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(3, 13),
        )
        
        self.fc1 = nn.Linear(13, 1)
        
    def forward(self, x):
         x1 = F.leaky_relu(self.fclayers(x))
         #print(x1.shape)
         x2 = self.convlayer(x)
         #print(x2.shape)
            
         x3 = x1+x2
         x3 = self.fc1(x3)
         return x3

# Data transform

In [206]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [207]:
# X_shape(X) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def X_shape(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    X_tensor = torch.split(X_tensor,1)
    X_tensor = torch.stack(X_tensor)
    print("X now in shape of",X_tensor.shape)
    return X_tensor

# y_tensor(y) : used for split the data into a right tensor size
#input: X in shape of 16760 rows × 7488 columns (df)
#output: X in shape of torch.Size([16760, 312, 24]) (tensor)
def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32).reshape(len(y),1)
    print("y now in shape of",y.shape)
    return y

In [208]:
# X should be in smaples, 1, squence, rows
X1 = X_shape(X_train_selected_t2_norm)
y1 = y_tensor(y_train_t2.iloc[:,-1])

train_datasets = CusDatasetLoader(X1, y1)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

x,y = train_datasets[0]
print(x.shape)
print(y.shape)

X now in shape of torch.Size([16760, 1, 54])
y now in shape of torch.Size([16760, 1])
torch.Size([1, 54])
torch.Size([1])


# Model train

In [209]:
model = JusNN(input_size, num_layers, output_size).to(device)

In [210]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

criterion = RMSELoss
optimizer = optim.Adadelta(model.parameters(), lr = learning_rate)

In [211]:
losses=[]
acc_list=[]
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.permute(0,1,2)
        #print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        targets = torch.squeeze(targets, 1)
        #print(targets)
        
        # forward
        scores = model(data)
        #print(scores.shape)
        loss = criterion(scores, targets)
        l2_lambda = l2_lamb
        l2_norm = sum(p.pow(2.0).sum()
                  for p in model.parameters())
 
        loss = loss + l2_lambda * l2_norm
    
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
    if epoch%5 == 0:
        losses.append(loss.detach().numpy() )
        print(f'Epoch {epoch:03}: | Loss: {loss:.5f} ')
        

Epoch 000: | Loss: 4.58264 
Epoch 005: | Loss: 2.50964 
Epoch 010: | Loss: 1.98793 
Epoch 015: | Loss: 2.27600 
Epoch 020: | Loss: 1.46798 
Epoch 025: | Loss: 1.21482 
Epoch 030: | Loss: 1.74973 
Epoch 035: | Loss: 2.02878 
Epoch 040: | Loss: 1.83034 
Epoch 045: | Loss: 2.01138 
Epoch 050: | Loss: 1.42939 
Epoch 055: | Loss: 1.77593 
Epoch 060: | Loss: 2.51421 
Epoch 065: | Loss: 2.27796 
Epoch 070: | Loss: 2.11736 
Epoch 075: | Loss: 1.68948 
Epoch 080: | Loss: 1.72143 
Epoch 085: | Loss: 1.23671 
Epoch 090: | Loss: 1.91781 
Epoch 095: | Loss: 1.91700 
Epoch 100: | Loss: 1.59808 
Epoch 105: | Loss: 1.78603 
Epoch 110: | Loss: 2.07805 
Epoch 115: | Loss: 1.46868 
Epoch 120: | Loss: 2.65344 
Epoch 125: | Loss: 2.23157 
Epoch 130: | Loss: 2.23309 
Epoch 135: | Loss: 1.48596 
Epoch 140: | Loss: 1.78986 
Epoch 145: | Loss: 1.71603 
Epoch 150: | Loss: 1.93434 
Epoch 155: | Loss: 2.60820 
Epoch 160: | Loss: 1.58716 
Epoch 165: | Loss: 1.80276 
Epoch 170: | Loss: 2.05713 
Epoch 175: | Loss: 2

# Model RMSE

In [212]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def check_rmse(loader, model):

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for data, targets in loader:
            data = data.permute(0,1,2)
            #print(data.shape)
            data = data.to(device=device)
            targets = targets.to(device=device)
            targets = torch.squeeze(targets, 1)

            scores = model(data)
            res = rmse(scores,targets)
        
            
    # Toggle model back to train
    model.train()
    return res
    

In [213]:
X2 = X_shape(X_valid_selected_t2_norm)
y2 = y_tensor(y_valid_t2.iloc[:,-1])

Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
print(f"RMSE on test set: {check_rmse(Valid_loader, model)}")

X now in shape of torch.Size([2394, 1, 54])
y now in shape of torch.Size([2394, 1])
RMSE on test set: 1.9922609329223633


# Test

In [214]:
def test(loader, model):
    # Set model to eval
    model.eval()
    res = torch.tensor([], dtype=torch.int64)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)
            print(y.shape)

            scores = model(x)
            _, predictions = scores.max(1)
            res =  torch.cat((predictions, res), 0)  
    model.train()
   
    return res

In [ ]:
X_test_final = X_shape(X_test)
y_final = torch.zeros([4790,1])

test_datasets = CusDatasetLoader(X_test_final, y_final)
test_loader = DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=True)

result = test(test_loader, model)
result

In [ ]:

#df.index = ['Row_1', 'Row_2', 'Row_3', 'Row_4']
res1 = pd.DataFrame(result, columns=["Predicted"])
res1.index = X_test.index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out.csv')


#

#

#